**Ritajit Majumdar**<br>
Enabling Technologies Researcher @ IBM Quantum<br>
majumdar.ritajit@ibm.com

## SPAM calibration via measurement twirling

State preparation and measurement (SPAM) error implies the scenario where the outcome of the state is $q \in \{0,1\}$, but it is incorrectly recorded as $q \oplus 1$ with probability $p$. The value of $p$ differs from qubit to qubit, and varies over time as well.<br>

In this notebook we shall calculate the SPAM fidelity of each qubit on a device by twirling the measurement, and then finding the expectation value of weight-1 $Z$ observable on that qubit. In other words, we

<ol>
    <li>Twirl each qubit $k$ times. In this method</li>
    <ol>
        <li>a $X$ gate is applied randomly prior to the measurement.</li>
        <li>the measurement outcome is recorded.</li>
        <li>if a $X$ gate was applied, then the measurement outcome is inverted to account for it.</li>
    </ol>
    <li>The ideal expectation value of $\langle Z \rangle$ on each qubit should be +1.</li>
    <li>The obtained expectation value provides the SPAM fidelity of the qubit.</li>
</ol>

## Qiskit Pattern

### Step 1: Map problem to quantum hardware native format

In [13]:
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService()  # Credentials may be needed
backend = service.get_backend('ibm_kyiv')  # Choose your desired backend

In [27]:
num_qubits = backend.num_qubits

The calibration circuit is simply a circuit where each qubit is initialized in $|0\rangle$ and is measured. The ideal outcome of each qubit is, therefore, $0$ and hence the expectation value of $\langle Z \rangle$ is `+1` for each qubit.

In [15]:
from qiskit import QuantumCircuit

calibration_circuit = QuantumCircuit(num_qubits)
calibration_circuit.measure_all()

In [17]:
calibration_circuit.draw('mpl', fold=-1)

We define $\langle Z \rangle$ observable on each qubit. Note that we require the expectation value of each of these observables separately. Therefore, we create an array containing each of these observables.

In [28]:
from qiskit.quantum_info import SparsePauliOp

observable = [SparsePauliOp('I'*i + 'Z' + 'I'*(num_qubits-i-1)) for i in range(num_qubits)]

### Step 2: Optimize circuit and observable

In order to execute the logical circuit on actual hardware we need to express it in terms of the hardware's native gates (i.e. Instruction Set Architecture, ISA), as well as assign physical qubits for execution. This action is carried out through the process of circuit transpilation.

The calibration circuit is already hardware-native, and the placement is trivial since it encompasses the entire backend. However, we still need to define the placement via transpilation.

In [18]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

pm = generate_preset_pass_manager(optimization_level=3, backend=backend)

In [19]:
isa_calibration_circuit = pm.run(calibration_circuit)

Since the observables are already mapped to the qubits, we need not apply the circuit layout to the observables in this example.

### Step 3: Execute using Qiskit Primitives

We shall use the `EstimatorV2` primitive to run the circuit. We turn off all other error mitigation techniques by setting `resilience_level` to $0$.

In [21]:
from qiskit_ibm_runtime import EstimatorOptions

options = EstimatorOptions()

options.default_shots = 4096
options.twirling.enable_gates = False # gate twirling is disabled
options.dynamical_decoupling.enable = False # dynamical decoupling is disabled
options.resilience_level = 0

Now we turn on measurement twirling and the number of twirls is set to `32` which is the default value.

In [30]:
from qiskit_ibm_runtime import EstimatorV2

estimator = EstimatorV2(backend=backend, options=options)
pub = (isa_calibration_circuit, observable)

# apply measurement twirling
estimator.options.twirling.enable_measure = True
estimator.options.twirling.num_randomizations = 32 # this is the default value
job = estimator.run([pub])

In [32]:
print(f"JOB ID: {job.job_id()}")

JOB ID: cscbbpedape00085vdb0


### Step 4: Postprocess

Here we obtain the result and store the expectation value of each qubit separately. Note again that this expectation value provides us with the SPAM fidelity of each qubit.

In [34]:
result = job.result()[0]

In [37]:
spam_fidelity = {idx: result.data.evs[idx] for idx in range(num_qubits)}

In [38]:
spam_fidelity

{0: 0.98486328125,
 1: 0.9912109375,
 2: 0.98583984375,
 3: 0.8076171875,
 4: 0.99755859375,
 5: 0.8076171875,
 6: 0.99072265625,
 7: 0.99169921875,
 8: 0.9921875,
 9: 0.9521484375,
 10: 0.97119140625,
 11: 0.9912109375,
 12: 0.970703125,
 13: 0.97314453125,
 14: 0.9072265625,
 15: 0.9873046875,
 16: 0.98828125,
 17: 0.96435546875,
 18: 0.88037109375,
 19: 0.97265625,
 20: 0.7490234375,
 21: 0.974609375,
 22: 0.9736328125,
 23: 0.9931640625,
 24: 0.96044921875,
 25: 0.99560546875,
 26: 0.99072265625,
 27: 0.9697265625,
 28: 0.984375,
 29: 0.99462890625,
 30: 0.986328125,
 31: 0.97900390625,
 32: 0.962890625,
 33: 0.96435546875,
 34: 0.9892578125,
 35: 0.99072265625,
 36: 0.99365234375,
 37: 0.92529296875,
 38: 0.974609375,
 39: 0.9453125,
 40: 0.888671875,
 41: 0.98681640625,
 42: 0.98095703125,
 43: 0.93896484375,
 44: 0.970703125,
 45: 0.990234375,
 46: 0.71435546875,
 47: 0.95947265625,
 48: 0.99169921875,
 49: 0.9765625,
 50: 0.9931640625,
 51: 0.9912109375,
 52: 0.83740234375,
 53

### Practice

Challenge questions to come